In [4]:
from __future__ import division, print_function

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, UpSampling2D, Conv2D, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1   #imagens passadas em grayscale com apenas um canal
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100   #dimensao do espaco latente (ruido)
        
        optimizer = Adam(0.0002, 0.5)   # taxa de aprendizado e beta1 do otimizador Adam
        
        # Criando e compilando o discriminador
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = ['binary_crossentropy'], 
                                  optimizer = optimizer,
                                  metrics = ['accuracy'])
        
        # Construindo e compilando o gerador
        self.generator = self.build_generator()
        
        # Gerador recebe ruido na entrada e gera imagens
        z = Input(shape=(self.latent_dim))
        img = self.generator(z)
        
        # Somente o gerador e treinado no modelo da GAN
        self.discriminator.trainable = False
        
        # O discriminador recebe imagens geradas e determina sua validade
        validity = self.discriminator(img)
        
        # O modelo combinado treina o gerador com base na resposta do discriminador
        self.combined = Model(z, validity)
        self.combined.compile(loss = 'binary_crossentropy',
                             optimizer = optimizer)
        
        
    def build_generator(self):
        
        model = Sequential()
        
        model.add(Dense(128*7*7, activation='relu', input_dim=self.latent_dim))
        model.add(Reshape((7,7,128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation('tanh'))
        model.add(Reshape(self.img_shape))
        
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise, img)
    
    
    def build_discriminator(self):
        
        model = Sequential()
        
        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding='same'))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.summary()
                
        img = Input(shape=self.img_shape)
        
        features = model(img)
        validity = Dense(1, activation='sigmoid')(features)
        
        return Model(img, validity)
    
    
    def train(self, epochs, batch_size=128, sample_interval=50):
        
        (X_train, _),(_, _) = mnist.load_data()
        
        # Reescalando dados de [0 255] (greyscale) para [-1 1]
        X_train = (X_train - 127.5)/127.5
        X_train = np.expand_dims(X_train, axis=3)  #aumento de dimensao para casar com camadas convolucionais do modelo
        
        # Labels para falsos e verdadeiros
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):
            
            # Treinamento do Discriminador
            
            # Seleciona um conjunto aleatorio de imagens
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            # Gera um conjunto de novas imagens
            gen_imgs = self.generator.predict(noise)
            
            # Treina o discriminador
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5*np.add(d_loss_fake, d_loss_real)
            
            # Treina Gerador
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            # Treina Gerador (usa indices como se dados criados fossem verdadeiros para induzir erro no discriminador)
            g_loss = self.combined.train_on_batch(noise, valid)
            
            # Plot do progresso do treinamento
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            
            # Salvando imagens geradas com certo intervalo de epocas
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.save_model()

            
    def sample_images(self, epoch):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r*c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        
        # Reescalando imagens para [0 1]
        gen_imgs = 0.5*gen_imgs + 0.5
        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt+=1
        fig.savefig("images_gan_conv/%d.png" % epoch)
        plt.close()
        
        
    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model_gan_conv/%s.json" % model_name
            weights_path = "saved_model_gan_conv/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator")
        save(self.discriminator, "discriminator")
        
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=14000, batch_size=32, sample_interval=200)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 8, 8, 32)         

W0825 12:57:24.936630 19316 training.py:2197] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
W0825 12:57:26.673518 19316 training.py:2197] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?


0 [D loss: 0.792810, acc.: 50.00%] [G loss: 0.621585]
1 [D loss: 0.833369, acc.: 53.12%] [G loss: 0.556557]
2 [D loss: 0.614525, acc.: 67.19%] [G loss: 0.627576]
3 [D loss: 0.445845, acc.: 81.25%] [G loss: 0.726869]
4 [D loss: 0.349729, acc.: 90.62%] [G loss: 0.625614]
5 [D loss: 0.227564, acc.: 98.44%] [G loss: 0.536005]
6 [D loss: 0.208486, acc.: 98.44%] [G loss: 0.722581]
7 [D loss: 0.163741, acc.: 100.00%] [G loss: 0.585306]
8 [D loss: 0.126682, acc.: 100.00%] [G loss: 0.715402]
9 [D loss: 0.182411, acc.: 98.44%] [G loss: 0.528373]
10 [D loss: 0.244216, acc.: 92.19%] [G loss: 0.603580]
11 [D loss: 0.353256, acc.: 81.25%] [G loss: 0.595696]
12 [D loss: 0.583028, acc.: 64.06%] [G loss: 0.584646]
13 [D loss: 0.535879, acc.: 65.62%] [G loss: 0.724735]
14 [D loss: 0.653641, acc.: 59.38%] [G loss: 0.853752]
15 [D loss: 0.538066, acc.: 68.75%] [G loss: 0.885273]
16 [D loss: 0.661037, acc.: 64.06%] [G loss: 0.874790]
17 [D loss: 0.517453, acc.: 70.31%] [G loss: 1.098467]
18 [D loss: 0.4712

149 [D loss: 0.635554, acc.: 62.50%] [G loss: 1.167758]
150 [D loss: 0.792654, acc.: 56.25%] [G loss: 1.035456]
151 [D loss: 0.714359, acc.: 57.81%] [G loss: 1.211798]
152 [D loss: 0.828655, acc.: 53.12%] [G loss: 1.297639]
153 [D loss: 0.733064, acc.: 54.69%] [G loss: 1.153641]
154 [D loss: 0.650544, acc.: 68.75%] [G loss: 1.292324]
155 [D loss: 0.713230, acc.: 59.38%] [G loss: 0.885904]
156 [D loss: 0.874556, acc.: 48.44%] [G loss: 1.122058]
157 [D loss: 0.835334, acc.: 48.44%] [G loss: 1.031849]
158 [D loss: 0.816108, acc.: 54.69%] [G loss: 0.955693]
159 [D loss: 0.761669, acc.: 56.25%] [G loss: 1.158272]
160 [D loss: 0.822220, acc.: 53.12%] [G loss: 0.986984]
161 [D loss: 0.896558, acc.: 50.00%] [G loss: 1.107570]
162 [D loss: 0.768008, acc.: 48.44%] [G loss: 0.936896]
163 [D loss: 0.791314, acc.: 51.56%] [G loss: 1.177005]
164 [D loss: 0.893665, acc.: 40.62%] [G loss: 1.170949]
165 [D loss: 0.844814, acc.: 45.31%] [G loss: 1.009638]
166 [D loss: 0.842099, acc.: 48.44%] [G loss: 0.

296 [D loss: 0.650485, acc.: 67.19%] [G loss: 0.849337]
297 [D loss: 0.717284, acc.: 57.81%] [G loss: 0.850303]
298 [D loss: 0.804795, acc.: 48.44%] [G loss: 0.875810]
299 [D loss: 0.709537, acc.: 53.12%] [G loss: 0.854556]
300 [D loss: 0.715189, acc.: 60.94%] [G loss: 0.855416]
301 [D loss: 0.723116, acc.: 48.44%] [G loss: 0.923574]
302 [D loss: 0.661502, acc.: 60.94%] [G loss: 0.777580]
303 [D loss: 0.704907, acc.: 53.12%] [G loss: 0.858899]
304 [D loss: 0.830538, acc.: 39.06%] [G loss: 0.948358]
305 [D loss: 0.676614, acc.: 57.81%] [G loss: 1.108836]
306 [D loss: 0.653026, acc.: 62.50%] [G loss: 0.869194]
307 [D loss: 0.649365, acc.: 65.62%] [G loss: 0.874809]
308 [D loss: 0.733505, acc.: 64.06%] [G loss: 0.978019]
309 [D loss: 0.795726, acc.: 42.19%] [G loss: 0.964214]
310 [D loss: 0.675286, acc.: 59.38%] [G loss: 0.930869]
311 [D loss: 0.736188, acc.: 51.56%] [G loss: 1.034915]
312 [D loss: 0.706118, acc.: 54.69%] [G loss: 0.899712]
313 [D loss: 0.668812, acc.: 57.81%] [G loss: 0.

...

13366 [D loss: 0.757398, acc.: 39.06%] [G loss: 0.843030]
13367 [D loss: 0.641756, acc.: 64.06%] [G loss: 0.841670]
13368 [D loss: 0.681767, acc.: 50.00%] [G loss: 0.765689]
13369 [D loss: 0.689268, acc.: 56.25%] [G loss: 0.672536]
13370 [D loss: 0.658460, acc.: 62.50%] [G loss: 0.693905]
13371 [D loss: 0.771060, acc.: 46.88%] [G loss: 0.778587]
13372 [D loss: 0.712888, acc.: 53.12%] [G loss: 0.748134]
13373 [D loss: 0.681286, acc.: 51.56%] [G loss: 0.743694]
13374 [D loss: 0.658362, acc.: 60.94%] [G loss: 0.773853]
13375 [D loss: 0.680658, acc.: 54.69%] [G loss: 0.794128]
13376 [D loss: 0.659342, acc.: 59.38%] [G loss: 0.773032]
13377 [D loss: 0.754101, acc.: 39.06%] [G loss: 0.870724]
13378 [D loss: 0.704902, acc.: 48.44%] [G loss: 0.846615]
13379 [D loss: 0.651800, acc.: 62.50%] [G loss: 0.895947]
13380 [D loss: 0.661045, acc.: 57.81%] [G loss: 0.804702]
13381 [D loss: 0.660626, acc.: 57.81%] [G loss: 1.002803]
13382 [D loss: 0.677078, acc.: 60.94%] [G loss: 0.869518]
13383 [D loss:

13508 [D loss: 0.663715, acc.: 60.94%] [G loss: 0.790750]
13509 [D loss: 0.640605, acc.: 65.62%] [G loss: 0.851603]
13510 [D loss: 0.697356, acc.: 42.19%] [G loss: 0.814167]
13511 [D loss: 0.706222, acc.: 50.00%] [G loss: 0.821616]
13512 [D loss: 0.715500, acc.: 54.69%] [G loss: 0.760262]
13513 [D loss: 0.685503, acc.: 53.12%] [G loss: 0.762910]
13514 [D loss: 0.716591, acc.: 51.56%] [G loss: 0.778466]
13515 [D loss: 0.675814, acc.: 65.62%] [G loss: 0.824011]
13516 [D loss: 0.655271, acc.: 65.62%] [G loss: 0.744544]
13517 [D loss: 0.673401, acc.: 65.62%] [G loss: 0.726698]
13518 [D loss: 0.647343, acc.: 60.94%] [G loss: 0.895122]
13519 [D loss: 0.631595, acc.: 65.62%] [G loss: 0.778205]
13520 [D loss: 0.667713, acc.: 62.50%] [G loss: 0.866866]
13521 [D loss: 0.632852, acc.: 62.50%] [G loss: 0.816229]
13522 [D loss: 0.654676, acc.: 64.06%] [G loss: 0.846612]
13523 [D loss: 0.709177, acc.: 53.12%] [G loss: 0.755748]
13524 [D loss: 0.662309, acc.: 62.50%] [G loss: 0.797150]
13525 [D loss:

13650 [D loss: 0.646757, acc.: 65.62%] [G loss: 0.752385]
13651 [D loss: 0.709169, acc.: 48.44%] [G loss: 0.805258]
13652 [D loss: 0.733452, acc.: 53.12%] [G loss: 0.822659]
13653 [D loss: 0.674377, acc.: 57.81%] [G loss: 0.805701]
13654 [D loss: 0.631917, acc.: 64.06%] [G loss: 0.745071]
13655 [D loss: 0.697274, acc.: 54.69%] [G loss: 0.745026]
13656 [D loss: 0.704071, acc.: 54.69%] [G loss: 0.711164]
13657 [D loss: 0.642077, acc.: 64.06%] [G loss: 0.800391]
13658 [D loss: 0.767426, acc.: 46.88%] [G loss: 0.847274]
13659 [D loss: 0.670672, acc.: 54.69%] [G loss: 0.778489]
13660 [D loss: 0.660636, acc.: 65.62%] [G loss: 0.832367]
13661 [D loss: 0.687457, acc.: 57.81%] [G loss: 0.798600]
13662 [D loss: 0.705811, acc.: 51.56%] [G loss: 0.760695]
13663 [D loss: 0.676351, acc.: 57.81%] [G loss: 0.740317]
13664 [D loss: 0.726523, acc.: 45.31%] [G loss: 0.745336]
13665 [D loss: 0.675393, acc.: 62.50%] [G loss: 0.805935]
13666 [D loss: 0.717929, acc.: 48.44%] [G loss: 0.774310]
13667 [D loss:

13792 [D loss: 0.702180, acc.: 51.56%] [G loss: 0.764732]
13793 [D loss: 0.715908, acc.: 45.31%] [G loss: 0.843660]
13794 [D loss: 0.651240, acc.: 57.81%] [G loss: 0.761783]
13795 [D loss: 0.713034, acc.: 53.12%] [G loss: 0.832543]
13796 [D loss: 0.642930, acc.: 64.06%] [G loss: 0.831534]
13797 [D loss: 0.665732, acc.: 53.12%] [G loss: 0.809685]
13798 [D loss: 0.654932, acc.: 59.38%] [G loss: 0.813430]
13799 [D loss: 0.672962, acc.: 57.81%] [G loss: 0.837376]
13800 [D loss: 0.626092, acc.: 65.62%] [G loss: 0.715594]
13801 [D loss: 0.665652, acc.: 60.94%] [G loss: 0.792657]
13802 [D loss: 0.664348, acc.: 53.12%] [G loss: 0.734661]
13803 [D loss: 0.673970, acc.: 57.81%] [G loss: 0.824928]
13804 [D loss: 0.731456, acc.: 45.31%] [G loss: 0.914411]
13805 [D loss: 0.689348, acc.: 54.69%] [G loss: 0.831831]
13806 [D loss: 0.652764, acc.: 56.25%] [G loss: 0.815821]
13807 [D loss: 0.696209, acc.: 51.56%] [G loss: 0.797952]
13808 [D loss: 0.632070, acc.: 60.94%] [G loss: 0.838871]
13809 [D loss:

13934 [D loss: 0.713439, acc.: 45.31%] [G loss: 0.860562]
13935 [D loss: 0.702913, acc.: 54.69%] [G loss: 0.750461]
13936 [D loss: 0.669389, acc.: 56.25%] [G loss: 0.760873]
13937 [D loss: 0.697288, acc.: 51.56%] [G loss: 0.854802]
13938 [D loss: 0.607375, acc.: 71.88%] [G loss: 0.820038]
13939 [D loss: 0.723534, acc.: 57.81%] [G loss: 0.843260]
13940 [D loss: 0.591210, acc.: 68.75%] [G loss: 0.831870]
13941 [D loss: 0.674223, acc.: 57.81%] [G loss: 0.857980]
13942 [D loss: 0.622080, acc.: 68.75%] [G loss: 0.719398]
13943 [D loss: 0.682843, acc.: 57.81%] [G loss: 0.772106]
13944 [D loss: 0.617377, acc.: 68.75%] [G loss: 0.848515]
13945 [D loss: 0.651759, acc.: 59.38%] [G loss: 0.805982]
13946 [D loss: 0.698028, acc.: 54.69%] [G loss: 0.738637]
13947 [D loss: 0.673243, acc.: 57.81%] [G loss: 0.853764]
13948 [D loss: 0.707265, acc.: 54.69%] [G loss: 0.727286]
13949 [D loss: 0.729911, acc.: 53.12%] [G loss: 0.831520]
13950 [D loss: 0.746430, acc.: 42.19%] [G loss: 0.858056]
13951 [D loss: